In [2]:
# Import the necessary Packages for this software to run
import mediapipe as mp
import cv2
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression

In [3]:
file_path = 'val_sortiert_new_format.csv'
df = pd.read_csv(file_path)

In [4]:
#split into target and data
target = df['label_id']
data = df.iloc[:,3:]
print("data: ",data.shape)
print("label:", target.shape)

#filter dataframe
filtered_df = df[df['label_id'] != 20]

#split into target and data
target = filtered_df['label_id']
data = filtered_df.iloc[:,3:]

#split into train and test data
X_train, X_test, y_train, y_test = train_test_split(data,target,random_state=1,test_size=0.2,shuffle=True)

data:  (2233, 2331)
label: (2233,)


In [5]:
#Create Model
clr_filtered = LogisticRegression(max_iter=2000, random_state=1)
clr_filtered.fit(X_train, y_train)

#predict training and new data
y_train_pred = clr_filtered.predict(X_train) 
y_test_pred = clr_filtered.predict(X_test)

#evaluation of the prediction
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Training Set Accuracy:", train_accuracy)
print("Test Set Accuracy:", test_accuracy)

Training Set Accuracy: 0.8045112781954887
Test Set Accuracy: 0.7042606516290727


In [6]:
#Create Table
df_columns = [] #'video_id', 'label_id', 'label']
for frame in range(1,38):
    for landmark in range (0,21):
        s1 = f'{"F"}{frame}{"_L"}{landmark}{"_X"}'
        s2 = f'{"F"}{frame}{"_L"}{landmark}{"_Y"}'
        s3 = f'{"F"}{frame}{"_L"}{landmark}{"_Z"}'
        df_columns.append(s1)
        df_columns.append(s2)
        df_columns.append(s3)


In [7]:
#Create Dataframe
df = pd.DataFrame(columns=df_columns)

In [8]:
#Show Dataframe
df.head()

,F1_L0_X,F1_L0_Y,F1_L0_Z,F1_L1_X,F1_L1_Y,F1_L1_Z,F1_L2_X,F1_L2_Y,F1_L2_Z,F1_L3_X,...,F37_L17_Z,F37_L18_X,F37_L18_Y,F37_L18_Z,F37_L19_X,F37_L19_Y,F37_L19_Z,F37_L20_X,F37_L20_Y,F37_L20_Z


In [9]:
# Use MediaPipe to draw the hand framework over the top of hands it identifies
drawingModule = mp.solutions.drawing_utils
handsModule = mp.solutions.hands


def extract_data(image):
    # Use MediaPipe hand tracking with static_image_mode set to True
    with handsModule.Hands(static_image_mode=True, min_detection_confidence=0.7, max_num_hands=1) as hands:
        new_row=[]
        j = 0
        frame = cv2.resize(image, (640, 480))
        
        # Process the image and produce the hand framework overlay on top of the hand
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # In case the system sees multiple hands, this if statement deals with that and produces another hand overlay
        if results.multi_hand_landmarks:
            for handLandmarks in results.multi_hand_landmarks:
                drawingModule.draw_landmarks(frame, handLandmarks, handsModule.HAND_CONNECTIONS)
                i = 0
                last_idx = 0
                for idx, landmark in enumerate(handLandmarks.landmark):
                    while (i < idx):
                        new_row.append(0)
                        new_row.append(0)
                        new_row.append(0)
                        i = i+1

                    new_row.append(landmark.x)
                    new_row.append(landmark.y)
                    new_row.append(landmark.z)
                    i = idx+1
                    last_idx = idx
                if last_idx <= 20:
                    for i in range(last_idx+1, 21):
                        new_row.append(0)
                        new_row.append(0)
                        new_row.append(0)
        
        else:
            for i in range (0,63):
                new_row.append(0)
            j = j+1
        result = new_row.copy()
        new_row.clear()
        return result

In [10]:
# images_dir = "C:\\Users\\liz\\Documents\\GitHub\\SoSe-2024\\BV-Projekt-Echtzeit-Handerkennung\\14zip\\14"
# images = []
# for image in os.listdir(images_dir):
#     image_path = os.path.join(images_dir, image)
#     images.append(cv2.imread(image_path))


cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
row_arr = []
counter = 0
n = 37

while True:
    ret, frame = cap.read()
    row_arr.extend(extract_data(frame))
    counter += 1
    if counter > n:
        del row_arr[0:63]
        counter -= 1
    if counter == n:
        # model anwenden
        df.loc[0] = row_arr
        y_prediction = clr_filtered.predict(df)
        probabilities = clr_filtered.predict_proba(df)
        probabilities_in_percent = np.round(probabilities * 100, 2)
        print(y_prediction)
        print(probabilities_in_percent)
    #Below shows the current frame to the desktop 
    cv2.imshow("Frame", frame)                                                                                                                                                                                                                                                                                                                                                                 
    key = cv2.waitKey(1) & 0xFF
    
    #Below states that if the |q| is press on the keyboard it will stop the system
    if key == ord("q"):
        break

[11]
[[ 6.85 14.2  17.99 16.98 11.74 17.18  8.36  6.71]]


C:\Users\liz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\liz\AppData\Local\Temp\ipykernel_18316\592330377.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.15257081389427185' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[0] = row_arr
C:\Users\liz\AppData\Local\Temp\ipykernel_18316\592330377.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9274910092353821' has dtype incompatible with int64, please explicitl

[19]
[[ 1.89 19.86 15.31  7.69  3.04 22.52  9.77 19.92]]


C:\Users\liz\AppData\Local\Temp\ipykernel_18316\592330377.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.15257081389427185' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[0] = row_arr
C:\Users\liz\AppData\Local\Temp\ipykernel_18316\592330377.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9274910092353821' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[0] = row_arr
C:\Users\liz\AppData\Local\Temp\ipykernel_18316\592330377.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.206565785760176e-07' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[0] = row_arr
C:\Users\liz\AppData\Local\

[19]
[[ 5.36  7.27  7.74 16.8   9.57 32.8  11.74  8.71]]


...

C:\Users\liz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\liz\AppData\Local\Temp\ipykernel_18316\592330377.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.15257081389427185' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[0] = row_arr
C:\Users\liz\AppData\Local\Temp\ipykernel_18316\592330377.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9274910092353821' has dtype incompatible with int64, please explicitl

[24]
[[ 0.86  0.33 15.37  0.24 15.15 13.65 38.81 15.59]]


C:\Users\liz\AppData\Local\Temp\ipykernel_18316\592330377.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.15257081389427185' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[0] = row_arr
C:\Users\liz\AppData\Local\Temp\ipykernel_18316\592330377.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9274910092353821' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[0] = row_arr
C:\Users\liz\AppData\Local\Temp\ipykernel_18316\592330377.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.206565785760176e-07' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[0] = row_arr
C:\Users\liz\AppData\Local\

[13]
[[1.510e+00 1.000e-02 3.000e-01 9.000e-02 9.415e+01 1.310e+00 4.000e-01
  2.230e+00]]


: 